# Análise do COVID-19 em SP (Infectados)

## 1. Base de Dados Original

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
plt.style.use('seaborn-whitegrid')

df = pd.read_csv("https://raw.githubusercontent.com/elhenrico/covid19-Brazil-timeseries/master/confirmed-cases.csv").fillna(0)
df.rename({"Unnamed: 0": "Estado", "Unnamed: 1": "Sigla"}, axis='columns', inplace=True)
df.head(10)

,Estado,Sigla,26/2,27/2,28/2,29/2,1/3,2/3,3/3,4/3,...,23/3,24/3,25/3,26/3,27/3,28/3,29/3,30/3,31/3,1/4
0,Brasil,BR,1,1,1,2,2,2,2,3,...,1891,2201,2433,2915,3417,3904,4256,4579,5717,6836
1,Acre,AC,0,0,0,0,0,0,0,0,...,11,17,23,24,25,25,34,42,42,43
2,Amapá,AP,0,0,0,0,0,0,0,0,...,1,1,1,2,2,4,4,8,10,11
3,Amazonas,AM,0,0,0,0,0,0,0,0,...,32,47,54,67,81,111,140,151,175,200
4,Pará,PA,0,0,0,0,0,0,0,0,...,5,5,7,13,13,17,18,21,32,40
5,Rondônia,RO,0,0,0,0,0,0,0,0,...,3,3,5,5,6,6,6,6,8,9
6,Roraima,RR,0,0,0,0,0,0,0,0,...,2,2,8,8,10,12,16,16,16,22
7,Tocantins,TO,0,0,0,0,0,0,0,0,...,5,7,7,7,8,9,9,10,11,12
8,Norte,(N),0,0,0,0,0,0,0,0,...,59,82,105,126,145,184,227,254,294,337
9,Alagoas,AL,0,0,0,0,0,0,0,0,...,7,9,11,11,11,14,17,17,18,18


## 2. Total de casos diários no Brasil (ordenação decrescente)

In [2]:
df = df[df.Estado != 'Sudeste']
df = df[df.Estado != 'Nordeste']
df = df[df.Estado != 'Centro-Oeste']
df = df[df.Estado != 'Sul']
df = df[df.Estado != 'Norte']
df = df[df.Estado != 'Brasil']

confirmedCasesDf = df.copy()
column_list = list(confirmedCasesDf)
column_last_day = column_list[-1:]
confirmedCasesDf["Casos Confirmados"] = confirmedCasesDf[column_last_day].astype('int32')

sortedConfirmedCasesGroupedDf = confirmedCasesDf.sort_values(by=['Casos Confirmados'], ascending = False)
sortedConfirmedCasesGroupedDf.head(10)

,Estado,Sigla,26/2,27/2,28/2,29/2,1/3,2/3,3/3,4/3,...,24/3,25/3,26/3,27/3,28/3,29/3,30/3,31/3,1/4,Casos Confirmados
22,São Paulo,SP,1,1,1,2,2,2,2,3,...,810,862,1052,1223,1406,1451,1517,2339,2981,2981
21,Rio de Janeiro,RJ,0,0,0,0,0,0,0,0,...,305,370,421,493,558,600,657,708,832,832
11,Ceará,CE,0,0,0,0,0,0,0,0,...,182,200,235,282,314,348,372,390,444,444
28,Distrito Federal,DF,0,0,0,0,0,0,0,0,...,160,160,200,230,260,289,312,332,355,355
20,Minas Gerais,MG,0,0,0,0,0,0,0,0,...,130,133,153,189,205,231,261,275,314,314
25,Rio Grande do Sul,RS,0,0,0,0,0,0,0,0,...,98,123,168,195,197,226,241,274,306,306
10,Bahia,BA,0,0,0,0,0,0,0,0,...,76,84,104,115,128,154,176,213,246,246
26,Santa Catarina,SC,0,0,0,0,0,0,0,0,...,107,109,122,149,184,194,197,219,235,235
24,Paraná,PR,0,0,0,0,0,0,0,0,...,65,81,102,119,133,148,155,179,224,224
3,Amazonas,AM,0,0,0,0,0,0,0,0,...,47,54,67,81,111,140,151,175,200,200


### 2.1 Total de casos diários somente em SP

In [3]:
confirmedCasesByDaySPDf = confirmedCasesDf[confirmedCasesDf.Estado == 'São Paulo']
confirmedCasesByDaySPDf

,Estado,Sigla,26/2,27/2,28/2,29/2,1/3,2/3,3/3,4/3,...,24/3,25/3,26/3,27/3,28/3,29/3,30/3,31/3,1/4,Casos Confirmados
22,São Paulo,SP,1,1,1,2,2,2,2,3,...,810,862,1052,1223,1406,1451,1517,2339,2981,2981


In [4]:
confirmedCasesByDaySPCleanDf = confirmedCasesByDaySPDf.drop(["Estado", "Sigla", "Casos Confirmados"], axis=1)
datesSP = list(confirmedCasesByDaySPCleanDf)

i = 0
for d in datesSP:
    if i < 9:
        datesSP[i] = '03/0' + str(i + 1) + '/2020'
    else:
        datesSP[i] = '03/' + str(i + 1) + '/2020'
    i = i + 1

infectedSP = confirmedCasesByDaySPCleanDf.iloc[0]

dfForPlotSP = pd.DataFrame(
    {'datas': datesSP, 'infectados': infectedSP})
dfForPlotSP['datas'] = pd.to_datetime(dfForPlotSP['datas'])
dfForPlotSP

DateParseError: Invalid date specified (3/32)

In [ ]:
fig = plt.figure()
ax = plt.axes()

left, right = plt.xlim()
plt.xlim(0, 0.01)

dfForPlotSP.set_index('datas')['infectados'].plot(legend=True, grid=True, label='Confirmed Cases');
plt.title("COVID-19 in São Paulo")
plt.xlabel("Dates")
plt.ylabel("Number of cases")
plt.legend()
left, right

## 3. Previsão por meio da Regressão

### 3.1 Dados comuns a todos os algoritmos abaixo

In [ ]:
def plotGraph(XTrained, yTrained, XReal, yReal, algorithm):
    df = pd.DataFrame(
    {'datas': X, 'infectados': y.astype(int)})
    df['datas'] = pd.to_datetime(df['datas'])

    fig = plt.figure()
    ax = plt.axes()
    redLine = plt.plot(XReal, yReal, 'r:', label='Confirmed Cases')
    df.set_index('datas')['infectados'].plot(legend=True, grid=True, label='Predicted Cases');
    plt.title("COVID-19 in São Paulo ("+algorithm+")")
    plt.xlabel("Datas")
    plt.ylabel("Número de casos")
    
    blue_patch = mpatches.Patch(color='blue', label='Predicted Cases')
    red_patch = mpatches.Patch(color='red', label='Confirmed Cases')
    plt.legend(handles=[red_patch, blue_patch])

In [ ]:
import datetime

diasASimular = 60
dfForPlotDaysWithInfected = dfForPlotSP[dfForPlotSP['infectados'] != 0]

XTrain = np.arange(0, len(dfForPlotDaysWithInfected['datas']))
yTrain = dfForPlotDaysWithInfected['infectados']

XPredict = np.arange(0, len(XTrain) + diasASimular)

XRealCases = dfForPlotDaysWithInfected['datas']
yRealCases = dfForPlotDaysWithInfected['infectados']

predictedDays = pd.date_range(dfForPlotDaysWithInfected['datas'].iloc[-1] + datetime.timedelta(days=1), periods=diasASimular, freq='D')
X = np.append(dfForPlotDaysWithInfected['datas'], predictedDays)

### 3.1 Regressão Linear

In [ ]:
#treino
from sklearn.linear_model import LinearRegression
LRModel = LinearRegression()
LRModel.fit(XTrain.reshape(-1, 1), yTrain)

#predição
y = LRModel.predict(XPredict.reshape(-1, 1)).astype(int)
y = y + y[0] * -1
y

In [ ]:
plotGraph(X, y, XRealCases, yRealCases, "Linear Regression")

### 3.2 Polynomial Regression 2D

In [ ]:
#treino
from sklearn.preprocessing import PolynomialFeatures
polynomialFeatures = PolynomialFeatures(degree = 2)
XPoly = polynomialFeatures.fit_transform(XTrain.reshape(-1, 1))

from sklearn.linear_model import LinearRegression
polyLinearRegression = LinearRegression()
polyLinearRegression.fit(XPoly, yTrain)

#predição
XPoly = polynomialFeatures.fit_transform(XPredict.reshape(-1, 1))
y = polyLinearRegression.predict(XPoly).astype(int)
miny = min(y)
y = y + miny * -1

In [ ]:
plotGraph(X, y, XRealCases, yRealCases, "Polynomial Linear Regression 2D")

### 3.3 Polynomial Regression 3D

In [ ]:
#treino
from sklearn.preprocessing import PolynomialFeatures
polynomialFeatures = PolynomialFeatures(degree = 3)
XPoly = polynomialFeatures.fit_transform(XTrain.reshape(-1, 1))

from sklearn.linear_model import LinearRegression
polyLinearRegression = LinearRegression()
polyLinearRegression.fit(XPoly, yTrain)

#predição
XPoly = polynomialFeatures.fit_transform(XPredict.reshape(-1, 1))
y = polyLinearRegression.predict(XPoly).astype(int)
miny = min(y)
y = y + miny * -1

In [ ]:
plotGraph(X, y, XRealCases, yRealCases, "Polynomial Linear Regression 3D")

### 3.4 Polynomial Regression 4D

In [ ]:
#treino
from sklearn.preprocessing import PolynomialFeatures
polynomialFeatures = PolynomialFeatures(degree = 4)
XPoly = polynomialFeatures.fit_transform(XTrain.reshape(-1, 1))

from sklearn.linear_model import LinearRegression
polyLinearRegression = LinearRegression()
polyLinearRegression.fit(XPoly, yTrain)

#predição
XPoly = polynomialFeatures.fit_transform(XPredict.reshape(-1, 1))
y = polyLinearRegression.predict(XPoly).astype(int)
miny = min(y)
y = y + miny * -1

In [ ]:
plotGraph(X, y, XRealCases, yRealCases, "Polynomial Linear Regression 4D")